## Setup

In [1]:
from pathlib import Path
from nanover.recording.old.reading import buffer_to_frame_message, MessageRecordingReader
from nanover.recording.old.writing import record_entries_to_file

In [2]:
def remove_keys_from_frame(frame_response, keys):
    for key in keys:
        if key in frame_response.frame.values:
            del frame_response.frame.values[key]
        if key in frame_response.frame.arrays:
            del frame_response.frame.arrays[key]
    return frame_response

In [3]:
def read_messages_from_trajectory(path):
    with open(path, "rb") as io:
        reader = MessageRecordingReader.from_io(io)
        for entry in reader:
            yield entry.timestamp, buffer_to_frame_message(entry.buffer)

## Config

In [4]:
from nanover.trajectory.frame_data import PARTICLE_FORCES, PARTICLE_VELOCITIES
keys_to_strip = { PARTICLE_FORCES, PARTICLE_VELOCITIES }

INPUT_TRAJ_PATH = Path("nanotube-example-recording.traj")
OUTPUT_TRAJ_PATH = INPUT_TRAJ_PATH.parent / Path(INPUT_TRAJ_PATH.stem + "-filtered.traj")

print("STRIP:", ", ".join(keys_to_strip), "\nINPUT:", INPUT_TRAJ_PATH, "\nOUTPUT:", OUTPUT_TRAJ_PATH)

STRIP: particle.forces, particle.velocities 
INPUT: nanotube-example-recording.traj 
OUTPUT: nanotube-example-recording-filtered.traj


## Run

In [5]:
# NBVAL_SKIP
frames = read_messages_from_trajectory(INPUT_TRAJ_PATH)
filtered = ((timestamp, remove_keys_from_frame(frame, keys_to_strip)) for (timestamp, frame) in frames)
record_entries_to_file(OUTPUT_TRAJ_PATH, filtered)